In [3]:
#!pip install surprise

  Stored in directory: C:\Users\Hayden\AppData\Local\pip\Cache\wheels\78\9c\3d\41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [6]:
import gzip
from collections import defaultdict
import random
import pandas as pd
import numpy as np
import string
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer as tfv
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import json
import seaborn as sns
import math
import matplotlib.pyplot as plt
from scipy import stats
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
import surprise
import warnings; warnings.simplefilter('ignore')

In [10]:
revs = pd.read_csv("ratings_small.csv")
revs

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [34]:
md = pd.read_csv("movies_metadata.csv")
md = md.dropna(subset = ["title"])

In [35]:
def parse_collection(x):
    if type(x) != type("str"):
        return np.nan
    else:
        #Don't bother trying to actually parse the JSON, huge waste of time
        return int(x[7:x.find(",")])
        if "Weekend at Bernie" in x:
            return 118221
        return json.loads(x.replace("'", '"').replace("None", '"None"'))["id"]

In [36]:
#Basic processing - convert strings to ints and unpack jsons
md["collection_id"] = md["belongs_to_collection"].apply(parse_collection)
md["budget"] = md["budget"].apply(lambda x: math.log(float(x) + 1, 10.0))
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
md['id'] = md['id'].astype(int)
md['revenue'] = md['revenue'].apply(lambda x: math.log(float(x) + 1, 10.0))
md['runtime'] = md['runtime'].astype(float)
md['vote_average'] = md['vote_average'].astype(float)
md['vote_count'] = md['vote_count'].astype(int)


In [39]:
#For 1-hot encoding languages: arbitrarily only grab languages that appear in over 1000 movies
lvc = md["original_language"].value_counts()
def transform_language(x):
    if x in lvc and lvc[x] > 1000:
        return x
    else:
        return "ot"

In [40]:
md["t_language"] = md["original_language"].apply(transform_language)
dum = pd.get_dummies(md["t_language"])
md = pd.concat([md, dum], axis=1) 
md

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,vote_average,vote_count,collection_id,t_language,de,en,fr,it,ja,ot
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",7.477121,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,7.7,5415,10194.0,en,0,1,0,0,0,0
1,False,NaN,7.812913,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,6.9,2413,NaN,en,0,1,0,0,0,0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.000000,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,6.5,92,119050.0,en,0,1,0,0,0,0
3,False,NaN,7.204120,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,6.1,34,NaN,en,0,1,0,0,0,0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0.000000,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,5.7,173,96871.0,en,0,1,0,0,0,0
5,False,NaN,7.778151,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,7.7,1886,NaN,en,0,1,0,0,0,0
6,False,NaN,7.763428,"[Comedy, Romance]",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,6.2,141,NaN,en,0,1,0,0,0,0
7,False,NaN,0.000000,"[Action, Adventure, Drama, Family]",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,5.4,45,NaN,en,0,1,0,0,0,0
8,False,NaN,7.544068,"[Action, Adventure, Thriller]",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,5.5,174,NaN,en,0,1,0,0,0,0
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",7.763428,"[Adventure, Action, Thriller]",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,6.6,1194,645.0,en,0,1,0,0,0,0


In [41]:
print(md["t_language"].value_counts())

en    32267
ot     6797
fr     2438
it     1529
ja     1349
de     1080
Name: t_language, dtype: int64


In [42]:
#Create a subset of movies for computational ease.
links_small = pd.read_csv('links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 32)

In [43]:
#combine tagline and description, then run tf-idf on it
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')
tf = tfv(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])


In [45]:
#load in credits and keywords, then merge them into md
# ###Note: this resets the earlier change we made to combine tagline and description.
#           There is absolutely no reason for that.
credits = pd.read_csv('credits.csv')
keywords = pd.read_csv('keywords.csv')
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')


In [52]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 35)

In [53]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [54]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan


In [55]:
smd['director'] = smd['crew'].apply(get_director)

In [56]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)


In [57]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [58]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))